In [2]:
!pip install -q \
  torch>=2.1.0 \
  transformers>=4.39.0 \
  appdirs \
  jsonpickle \
  filelock \
  h5py \
  nltk \
  dotmap \
  pytest


In [3]:
!pip install radgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.0/588.0 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for radgraph: filename=radgraph-0.1.18-py3-none-any.whl size=812635 sha256=d9374df3fd5c033f99d9409750255f5d1026ce0b036090ac335dfc69508f19ef
  Stored in directory: /root/.cache/pip/wheels/fb/3c/fb/214f5d5cdab2a0f9f0904fd81d7fd1134404100b4444554df8
Successfully built radgraph


In [4]:
import json
from radgraph import get_radgraph_processed_annotations, RadGraph

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [5]:
import pandas as pd
import ast
import re


In [6]:

df = pd.read_csv("/content/output.csv")

clean = (
    df.iloc[0:50, 0]
      .astype(str)
      .str.replace(r'^FINDINGS:\s*', '', regex=True)
      .str.replace(r"\s+", " ", regex=True)
      .str.strip()
      .reset_index(drop=True)
)

concepts = df.iloc[0:50, 1].reset_index(drop=True)

eval_df = pd.DataFrame({"report": clean, "concepts": concepts})
model_type = "modern-radgraph-xl"
radgraph = RadGraph(model_type=model_type)
reports = eval_df["report"].astype(str).tolist()

anno = []

for report in reports:
    annotations = radgraph([report])  # one report batch
    processed = get_radgraph_processed_annotations(annotations)  # dict with "processed_annotations"
    anno.append(processed)


Using device: cpu


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


modern-radgraph-xl.tar.gz:   0%|          | 0.00/579M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/radgraph/radgraph.py:105: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path=model_dir)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [7]:
print(anno[3])


{'processed_annotations': [{'observation': 'clear', 'observation_start_ix': [3], 'located_at': ['lungs'], 'located_at_start_ix': [[1]], 'tags': ['definitely present'], 'suggestive_of': None, 'observation_end_ix': [3]}, {'observation': 'normal', 'observation_start_ix': [13], 'located_at': ['cardiac contours', 'hilar contours', 'mediastinal contours'], 'located_at_start_ix': [[6, 11], [8, 11], [10, 11]], 'tags': ['definitely present'], 'suggestive_of': None, 'observation_end_ix': [13]}, {'observation': 'abnormality', 'observation_start_ix': [17], 'located_at': ['pleural'], 'located_at_start_ix': [[16]], 'tags': ['definitely absent'], 'suggestive_of': None, 'observation_end_ix': [17]}], 'radgraph_annotations': {'0': {'text': 'The lungs are clear . The cardiac , hilar and mediastinal contours are normal . No pleural abnormality is seen .', 'entities': {'1': {'tokens': 'lungs', 'label': 'Anatomy::definitely present', 'start_ix': 1, 'end_ix': 1, 'relations': []}, '2': {'tokens': 'clear', 'la

In [8]:
print(clean[3])

The lungs are clear.The cardiac, hilar and mediastinal contours are normal.No pleural abnormality is seen.


In [10]:
import spacy
from spacy.tokens import DocBin
from spacy.util import filter_spans

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
nlp = spacy.blank("en")  # tokenizer only
docbin = DocBin(store_user_data=False)

def map_label(rg_label: str) -> str:
    # rg_label like: "Observation::definitely absent" or "Anatomy::definitely present"
    typ, tag = rg_label.split("::", 1)
    typ = typ.strip()
    tag = tag.strip()

    if typ == "Anatomy":
        return "ANATOMY"

    if typ == "Observation":
        if "absent" in tag:
            return "OBS_ABSENT"
        if "uncertain" in tag or "possible" in tag:
            return "OBS_UNCERTAIN"
        return "OBS_PRESENT"

    return None  # ignore others if any

def add_one_example(radgraph_annotations_obj: dict):
    # Your structure: processed["radgraph_annotations"]["0"]["text"] and ["entities"]
    item = radgraph_annotations_obj["0"]
    text = item["text"]
    ents = item["entities"]

    doc = nlp.make_doc(text)
    spans = []

    for _, ent in ents.items():
        rg_label = ent["label"]
        spacy_label = map_label(rg_label)
        if spacy_label is None:
            continue

        start_ix = ent["start_ix"]
        end_ix   = ent["end_ix"]

        # convert token indices -> character offsets
        start_char = doc[start_ix].idx
        end_char   = doc[end_ix].idx + len(doc[end_ix])

        span = doc.char_span(start_char, end_char, label=spacy_label, alignment_mode="contract")
        if span is not None:
            spans.append(span)

    doc.ents = filter_spans(spans)  # remove overlaps safely
    docbin.add(doc)

# Build training set from your list `anno`
for processed in anno:
    add_one_example(processed["radgraph_annotations"])

docbin.to_disk("/content/radgraph_train.spacy")
print("Saved /content/radgraph_train.spacy")


In [9]:
{'processed_annotations': [
    {
    'observation': 'clear',
    'observation_start_ix': [3],
    'located_at': ['lungs'],
    'located_at_start_ix': [[1]],
    'tags': ['definitely present'],
    'suggestive_of': None,
    'observation_end_ix': [3]
    }, {
    'observation': 'normal',
    'observation_start_ix': [13],
    'located_at': ['cardiac contours', 'hilar contours', 'mediastinal contours'],
    'located_at_start_ix': [[6, 11], [8, 11], [10, 11]],
    'tags': ['definitely present'],
    'suggestive_of': None,
    'observation_end_ix': [13]
    }, {
    'observation': 'abnormality',
    'observation_start_ix': [17],
    'located_at': ['pleural'],
    'located_at_start_ix': [[16]],
    'tags': ['definitely absent'],
    'suggestive_of': None,
    'observation_end_ix': [17]}],

 'radgraph_annotations': {
     '0': {'text': 'The lungs are clear . The cardiac , hilar and mediastinal contours are normal . No pleural abnormality is seen .',
           'entities': {'1': {'tokens': 'lungs',
                              'label': 'Anatomy::definitely present',
                              'start_ix': 1,
                              'end_ix': 1,
                              'relations': []},
                        '2': {'tokens': 'clear',
                              'label': 'Observation::definitely present',
                              'start_ix': 3,
                              'end_ix': 3,
                              'relations': [['located_at', '1']]},
                        '3': {'tokens': 'cardiac',
                              'label': 'Anatomy::definitely present',
                              'start_ix': 6,
                              'end_ix': 6,
                              'relations': []},
                        '4': {'tokens': 'hilar',
                              'label': 'Anatomy::definitely present',
                              'start_ix': 8,
                              'end_ix': 8,
                              'relations': []},
                        '5': {'tokens': 'mediastinal',
                              'label': 'Anatomy::definitely present',
                              'start_ix': 10,
                              'end_ix': 10,
                              'relations': []},
                        '6': {'tokens': 'contours',
                              'label': 'Anatomy::definitely present',
                              'start_ix': 11,
                              'end_ix': 11,
                              'relations': [['modify', '3'], ['modify', '4'], ['modify', '5']]},
                        '7': {'tokens': 'normal',
                              'label': 'Observation::definitely present',
                              'start_ix': 13,
                              'end_ix': 13,
                              'relations': [['located_at', '3'], ['located_at', '4'], ['located_at', '5']]},
                        '8': {'tokens': 'pleural',
                              'label': 'Anatomy::definitely present',
                              'start_ix': 16,
                              'end_ix': 16,
                              'relations': []},
                        '9': {'tokens': 'abnormality',
                              'label': 'Observation::definitely absent',
                              'start_ix': 17,
                              'end_ix': 17,
                              'relations': [['located_at', '8']]}},
           'data_source': None, 'data_split': 'inference'}},
 'start_ix_to_label': {'1': 'Anatomy::definitely present', '3': 'Observation::definitely present', '6': 'Anatomy::definitely present', '8': 'Anatomy::definitely present', '10': 'Anatomy::definitely present', '11': 'Anatomy::definitely present', '13': 'Observation::definitely present', '16': 'Anatomy::definitely present', '17': 'Observation::definitely absent'},
 'radgraph_text': 'The lungs are clear . The cardiac , hilar and mediastinal contours are normal . No pleural abnormality is seen .'}




{'processed_annotations': [{'observation': 'clear',
   'observation_start_ix': [3],
   'located_at': ['lungs'],
   'located_at_start_ix': [[1]],
   'tags': ['definitely present'],
   'suggestive_of': None,
   'observation_end_ix': [3]},
  {'observation': 'normal',
   'observation_start_ix': [13],
   'located_at': ['cardiac contours',
    'hilar contours',
    'mediastinal contours'],
   'located_at_start_ix': [[6, 11], [8, 11], [10, 11]],
   'tags': ['definitely present'],
   'suggestive_of': None,
   'observation_end_ix': [13]},
  {'observation': 'abnormality',
   'observation_start_ix': [17],
   'located_at': ['pleural'],
   'located_at_start_ix': [[16]],
   'tags': ['definitely absent'],
   'suggestive_of': None,
   'observation_end_ix': [17]}],
 'radgraph_annotations': {'0': {'text': 'The lungs are clear . The cardiac , hilar and mediastinal contours are normal . No pleural abnormality is seen .',
   'entities': {'1': {'tokens': 'lungs',
     'label': 'Anatomy::definitely present',



{'processed_annotations': [
    {
    'observation': 'clear',
    'observation_start_ix': [3],
    'located_at': ['lungs'],
    'located_at_start_ix': [[1]],
    'tags': ['definitely present'],
    'suggestive_of': None,
    'observation_end_ix': [3]
    }, {
    'observation': 'normal',
    'observation_start_ix': [13],
    'located_at': ['cardiac contours', 'hilar contours', 'mediastinal contours'],
    'located_at_start_ix': [[6, 11], [8, 11], [10, 11]],
    'tags': ['definitely present'],
    'suggestive_of': None,
    'observation_end_ix': [13]
    }, {
    'observation': 'abnormality',
    'observation_start_ix': [17],
    'located_at': ['pleural'],
    'located_at_start_ix': [[16]],
    'tags': ['definitely absent'],
    'suggestive_of': None,
    'observation_end_ix': [17]}],

 'radgraph_annotations': {
     '0': {'text': 'The lungs are clear . The cardiac , hilar and mediastinal contours are normal . No pleural abnormality is seen .',
           'entities': {'1': {'tokens': 'lungs',
                              'label': 'Anatomy::definitely present',
                              'start_ix': 1,
                              'end_ix': 1,
                              'relations': []},
                        '2': {'tokens': 'clear',
                              'label': 'Observation::definitely present',
                              'start_ix': 3,
                              'end_ix': 3,
                              'relations': [['located_at', '1']]},
                        '3': {'tokens': 'cardiac',
                              'label': 'Anatomy::definitely present',
                              'start_ix': 6,
                              'end_ix': 6,
                              'relations': []},
                        '4': {'tokens': 'hilar',
                              'label': 'Anatomy::definitely present',
                              'start_ix': 8,
                              'end_ix': 8,
                              'relations': []},
                        '5': {'tokens': 'mediastinal',
                              'label': 'Anatomy::definitely present',
                              'start_ix': 10,
                              'end_ix': 10,
                              'relations': []},
                        '6': {'tokens': 'contours',
                              'label': 'Anatomy::definitely present',
                              'start_ix': 11,
                              'end_ix': 11,
                              'relations': [['modify', '3'], ['modify', '4'], ['modify', '5']]},
                        '7': {'tokens': 'normal',
                              'label': 'Observation::definitely present',
                              'start_ix': 13,
                              'end_ix': 13,
                              'relations': [['located_at', '3'], ['located_at', '4'], ['located_at', '5']]},
                        '8': {'tokens': 'pleural',
                              'label': 'Anatomy::definitely present',
                              'start_ix': 16,
                              'end_ix': 16,
                              'relations': []},
                        '9': {'tokens': 'abnormality',
                              'label': 'Observation::definitely absent',
                              'start_ix': 17,
                              'end_ix': 17,
                              'relations': [['located_at', '8']]}},
           'data_source': None, 'data_split': 'inference'}},
 'start_ix_to_label': {'1': 'Anatomy::definitely present', '3': 'Observation::definitely present', '6': 'Anatomy::definitely present', '8': 'Anatomy::definitely present', '10': 'Anatomy::definitely present', '11': 'Anatomy::definitely present', '13': 'Observation::definitely present', '16': 'Anatomy::definitely present', '17': 'Observation::definitely absent'},
 'radgraph_text': 'The lungs are clear . The cardiac , hilar and mediastinal contours are normal . No pleural abnormality is seen .'}


